# LOCALIZACIONES DEFINIDAS
posibles lugares bien situados

In [1]:
london_loc = "51.520624794291884, -0.0870022262169723"

In [2]:
singapore_loc = "1.2862863534171092, 103.84868192263733"

In [3]:
san_francisco_loc = "37.79787988216447, -122.41049370727525"

# API
importar librerias

In [4]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import pyjsonviewer
import geopandas 
import shapely
import pymongo as pym
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry as shg

# FUNCIONES DE LLAMADA A LA API

In [5]:
load_dotenv()
#Tenemos los tokens guardados en un .env en el visual studio code

True

In [6]:
tk_id = os.getenv("token_client_id")

In [7]:
tk_secret = os.getenv("token_client_secret")

In [8]:
git_tk = os.getenv("github_tk")

In [9]:
url = 'https://api.foursquare.com/v2/venues/explore'

In [10]:
#pyjsonviewer.view_data(json_data = LaVariableQueQuierasSinComillas)

In [11]:
def llamada_API(url, coordenadas, query):
    parametros = {
    "client_id": tk_id,
    "client_secret": tk_secret,
    "v": "20180323",
    "ll": f"{coordenadas}",
    "query": f"{query}",
    "limit": 200  
}
    response = requests.get(url, params=parametros).json()["response"]["groups"][0]["items"]
    return response

In [12]:
def escoger_datos(response):
    name = []
    type_ = []
    lat = []
    lng = []
    cc = []
    adress = []
    for i in response:
        name.append(i["venue"]["name"])
    for i in response:
        type_.append(i["venue"]["categories"][0]["name"])
    for i in response:
        lng.append(i["venue"]["location"]["lng"])
    for i in response:
        lat.append(i["venue"]["location"]["lat"])
    for i in response:
        cc.append(i["venue"]["location"]["cc"])
    for i in response:
        adress.append(i["venue"]["location"]["formattedAddress"][0])      
    response2 = [name , type_, cc, adress,  lat, lng ]

    return response2

In [13]:
def crear_df(response2):
    df = pd.DataFrame(response2).T
    df1 = df.rename(columns = {0:"Name", 1:"Type", 2:"Country", 3:"Adress", 4:"Latitude", 5:"Longitude", })
    return df1

In [14]:
def get_geometry(df):
    df2 = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.Latitude, df.Longitude))
    return df2

In [15]:
def get_loc(df):
    df3 = df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return pd.DataFrame(df3)

In [16]:
def df_final1(url, coordenadas, query):
    a  = llamada_API(url, coordenadas, query)
    b = escoger_datos(a)
    c = crear_df(b)
    d = get_geometry(c)
    d["location"] = get_loc(d)
    df_terminado = d[["Name","Type","Country", "Adress","Latitude","Longitude","location"]]
    return df_terminado

# FUNCTIONS TO GET CONDITIONS OF A LOCATION

# SCHOOLS

In [17]:
Schools = ["Nursery School","Elementary School","Middle School","High School"]
def get_schools(location, Schools):
    Nursery_sch = df_final1(url, location, Schools[0])
    Elementary_sch = df_final1(url, location, Schools[1])
    Middle_sch = df_final1(url, location, Schools[2])
    High_sch = df_final1(url, location, Schools[3])
    all_schools_bad_index = pd.concat([Nursery_sch, Elementary_sch, Middle_sch, High_sch])
    all_schools = all_schools_bad_index.reset_index(drop=True)
    return all_schools

# DESIGNING COMPANIES

In [18]:
def get_design_studio(location):
    DesignCo = df_final1(url, location, "Design Studio")
    return DesignCo

# TECH STARTUPS

In [19]:
def get_tech_startups(location):
    TechStaUp = df_final1(url, location, "Tech Startup")
    return TechStaUp
    

# LEISURE

In [20]:
leisure = ["Nightclub", "Piano Bar", "Pub", "Irish Pub"]
def get_leisure(location, leisure):
    Disco = df_final1(url, location, leisure[0])
    Piano_bar = df_final1(url, location, leisure[1])
    Pub = df_final1(url, location, leisure[2])
    irish_pub = df_final1(url, location, leisure[3])
    all_leisure_bad_index = pd.concat([Disco, Piano_bar, Pub, irish_pub])
    leisure = all_leisure_bad_index.reset_index(drop=True)
    return leisure

# TRAVELLING STATIONS

In [21]:
stations = ["Metro Station", "Light Rail Station", "Bus Station"  ]
def get_closeby_stations(location, stations):
    Metro = df_final1(url, location, stations[0])
    Rail_Station = df_final1(url, location, stations[1])
    Bus_Station = df_final1(url, location, stations[1])
    all_stations_bad_index = pd.concat([Metro, Rail_Station, Bus_Station])
    stations = all_stations_bad_index.reset_index(drop=True)
    return stations

# PARKS

In [22]:
def get_parks(location):
    Parks = df_final1(url, london_loc, "Park")
    return Parks

# DATAFRAME GLOBAL Y LIMPIO

In [25]:
def df_FINAL(location):
    a = get_schools(location, Schools)
    b = get_design_studio(location)
    c = get_tech_startups(location)
    d = get_leisure(location, leisure)
    e = get_closeby_stations(location,stations)
    f = get_parks(location)
    global_dataframe_bad_index = pd.concat([a,b,c,d,e,f])
    global_df_final = global_dataframe_bad_index.reset_index(drop=True)
    return global_df_final

In [28]:
a = df_FINAL(london_loc)

In [30]:
a.sample(10)

,Name,Type,Country,Adress,Latitude,Longitude,location
108,Lanyrd HQ,Tech Startup,GB,32-38 Scrutton Street,51.523215,-0.081803,"{'type': 'Point', 'coordinates': (51.523214917..."
247,Duke Of Somerset,Pub,GB,15 Little Somerset Street (Mansell St),51.513644,-0.073862,"{'type': 'Point', 'coordinates': (51.513644015..."
24,Shackelwell Primary School,Elementary School,GB,Shackelwell Row,51.551951,-0.071973,"{'type': 'Point', 'coordinates': (51.551950637..."
275,The Hoop and Grapes,Pub,GB,47 High St (Mansell St),51.5144,-0.074285,"{'type': 'Point', 'coordinates': (51.514400476..."
58,Chroma Studios,Design Studio,GB,86-90 Paul Street,51.525719,-0.083741,"{'type': 'Point', 'coordinates': (51.525718702..."
189,The Owl & Pussycat,Pub,GB,34 Redchurch St,51.524366,-0.0755,"{'type': 'Point', 'coordinates': (51.524366445..."
231,The Fox,Pub,GB,28 Paul St.,51.523358,-0.08433,"{'type': 'Point', 'coordinates': (51.523357650..."
214,Bricklayer's Arms,Pub,GB,63 Charlotte Rd. (at Rivington St.),51.526095,-0.081181,"{'type': 'Point', 'coordinates': (51.526095376..."
159,Club Reina,Nightclub,GB,85 Charterhouse St,51.519715,-0.101815,"{'type': 'Point', 'coordinates': (51.519714631..."
162,Shoreditch House,Social Club,GB,Ebor Street,51.523781,-0.075933,"{'type': 'Point', 'coordinates': (51.52378114,..."


# LONDON

####### Schools
ldn_Nursery_sch = df_final(url, london_loc, "Nursery School")
ldn_Elementary_sch = df_final(url, london_loc, "Elementary School")
ldn_Middle_sch = df_final(url, london_loc, "Middle School")
ldn_High_sch = df_final(url, london_loc, "High School")
####### Design Studio
ldn_DesignCo = df_final(url, london_loc, "Design Studio")
####### Technological StartUps
ldn_TechStaUp = df_final(url, london_loc, "Tech Startup")
####### London Leisure
ldn_Disco = df_final(url, london_loc, "Nightclub")
ldn_Piano_bar = df_final(url, london_loc, "Piano Bar")
ldn_Pub = df_final(url, london_loc, "Pub")
ldn_irish_pub = df_final(url, london_loc, "Irish Pub")
####### Travelling 
ldn_Metro = df_final(url, london_loc, "Metro Station")
ldn_Rail_Station = df_final(url, london_loc, "Light Rail Station")
ldn_Bus_Station = df_final(url, london_loc, "Bus Station")
####### Parks (For Dobby and maintenance guy)
ldn_Park = df_final(url, london_loc, "Park"))
